In [7]:
#pip3 install langchain==0.1.12
#pip3 install langchain-openai == 0.0.8

In [1]:
import yaml,openai,os

with open("chatgpt_key.yml","r") as file:
    api_creds = yaml.safe_load(file)

os.environ["OPENAI_API_KEY"] = api_creds['openai_key']

In [2]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

In [3]:
model = ChatOpenAI(model_name="gpt-3.5-turbo",temperature=0)

#### Create basic chain

In [4]:
PROMPT = "tell me a joke about {topic}"
prompt = ChatPromptTemplate.from_template(PROMPT)
chain = (
            prompt
            |
            model
        )

response = chain.invoke({"topic":"bears"})
print(response.content)


Why did the bear break up with his girlfriend? 

Because he couldn't bear the relationship any longer!


In [6]:
# run on multiple prompt

topics = [{'topic':'AI'},{'topic':'Statistics'}]
responses = chain.map().invoke(topics)

In [10]:
for response in responses:
    print(response.content)
    print("-------------------------------------------------------------")
    

Why did the AI break up with its computer? 

Because it found someone who could give it more bytes!
-------------------------------------------------------------
Why did the statistician go to the beach?

To work on his tan-gent!
-------------------------------------------------------------


##### No conversation history

In [12]:
prompt = ChatPromptTemplate.from_template("{query}")
basic_chain = (
            prompt
            |
            model
        )

In [14]:
response = basic_chain.invoke({"query":'What are the first 4 color in rainbow?'})
print(response.content)

The first 4 colors in a rainbow are red, orange, yellow, and green.


In [16]:
response = basic_chain.invoke({"query":'And other three?'})
print(response.content)

1. The Great Wall of China
2. The Colosseum in Rome, Italy
3. Machu Picchu in Peru


#### Add memory to build conversation chain

In [17]:
from langchain.memory import ConversationBufferWindowMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

In [18]:
# create prompt template
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Act as a helpful AI Assistant"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)
# k=3 stores last 3 conversations between you and ChatGPT
memory = ConversationBufferWindowMemory(k=3, return_messages=True)

In [19]:
memory.load_memory_variables({}) # shows the conversation history

{'history': []}

In [20]:
from operator import itemgetter
# creates the conversation chain
chain = (
    RunnablePassthrough.assign(
        history=RunnableLambda(memory.load_memory_variables)
        |
        itemgetter("history")
    )
    |
    prompt
    |
    model
)

In [21]:
user_input = {'input': 'What are the first four colors of a rainbow'}
response = chain.invoke(user_input)
response.content

'The first four colors of a rainbow are red, orange, yellow, and green.'

In [22]:
memory.load_memory_variables({})

{'history': []}

In [23]:
# remember to save your conversation to the memory
memory.save_context(user_input, {"output": response.content})
memory.load_memory_variables({}) # remembers the conversation history

{'history': [HumanMessage(content='What are the first four colors of a rainbow'),
  AIMessage(content='The first four colors of a rainbow are red, orange, yellow, and green.')]}

In [24]:
user_input = {'input': 'And the last 3?'}
response = chain.invoke(user_input) # uses history of the past conversation to give a better response
response.content

'The last three colors of a rainbow are blue, indigo, and violet.'

In [25]:
memory.save_context(user_input, {"output": response.content})
memory.load_memory_variables({})

{'history': [HumanMessage(content='What are the first four colors of a rainbow'),
  AIMessage(content='The first four colors of a rainbow are red, orange, yellow, and green.'),
  HumanMessage(content='And the last 3?'),
  AIMessage(content='The last three colors of a rainbow are blue, indigo, and violet.')]}